# Explorando datos mundiales de COVID-19

In [ ]:
%matplotlib notebook
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display,clear_output
import matplotlib.pyplot as plt

## Preparación de datos

1. Explore el archivo covid19_confirmados.csv y configure la función read_csv de pandas para importarlo como un DataFrame.
    - Utilice el nombre de país como índice principal
    - Elimine las columnas Lat y Long
    - Use TimeStamps para los "nombres de columna" asociados a fechas
    - Reduzca la columna Province/State, el DataFrame resultante debe tener los totales a nivel país

### Datos de casos confirmados

Primero, importamos los datos de casos confirmados a un dataframe

In [ ]:
df_confirmados = pd.read_csv('./data/covid19_confirmados.csv')
df_confirmados.head(5)

Las primeras cuatro columnas identifican el lugar y las 400 columnas siguientes corresponden al número de casos en ese lugar, durante 400 días consecutivos.

Algunos países tienen solo una fila, otros tienen varias, cada una correspondiente a una provincia o estado.

Luego, reduciremos la columna 'Province/State' y dejaremos una fila por país. Para facilitar esa reducción, crearemos un MultiIndex con las columnas 'Country/Region' y 'Province/State'.

In [ ]:
df_confirmados.set_index(keys=['Country/Region', 'Province/State'], drop=True,inplace=True)

In [ ]:
df_confirmados.head(5)

Pero antes de hacer esa reducción, ordenaremos un poco más el dataframe.

Eliminamos las columnas "Lat" y "Long":

In [ ]:
df_confirmados.drop(labels=['Lat', 'Long'], axis=1, inplace=True)

In [ ]:
df_confirmados.head(5)

Luego, reinterpretamos las etiquetas de fecha como Timestamps en lugar de simples objetos.

Para eso, le aplicamos a cada columna la función `pd.to_datetime`.

In [ ]:
df_confirmados.rename(mapper=pd.to_datetime, axis='columns', inplace=True);
df_confirmados.head(5)

Ahora sí reducimos el índice Province/State, para que quede solo una fila por país. Los valores para cada día serán la suma de casos dentro de las provincias para aquellos países en que habían varias provincias. Para los países que solo tenían una fila, el valor se mantiene. **TO DO**: Revisar que estas operaciones hayan quedado bien calculadas para ambos casos.

In [ ]:
df_confirmados = df_confirmados.groupby(by='Country/Region').aggregate(np.sum)

In [ ]:
df_confirmados.head(5)

### Datos de población por país

Importamos los datos, usando el país como índice principal.

Solo importamos los datos de nombre de país y población. No usaremos los otros.

In [ ]:
df_poblacion = pd.read_csv('./data/population_by_country_2020.csv',usecols=['Country (or dependency)','Population (2020)'],index_col='Country (or dependency)')

df_poblacion.head(5)

Ordenamos los países por orden alfabético.

In [ ]:
df_poblacion.sort_index(inplace=True)
df_poblacion.head(5)

Debemos encontrar las diferencias entre las etiquetas de los índices de los dataframes.

Primero, veamos los tamaños de ambos índices.

In [ ]:
print('n° filas df confirmados:', len(df_confirmados.index))
print('n° filas df población:', len(df_poblacion.index))
print('diferencia:', abs(len(df_confirmados.index) - len(df_poblacion.index)))

Ahora, veamos qué etiquetas están en el dataframe de confirmados pero no en el de población.

In [ ]:
df_confirmados.index.difference(df_poblacion.index)

In [ ]:
len(_)

Ahora lo contrario, es decir, cuáles están en población pero no en confirmados.

In [ ]:
df_poblacion.index.difference(df_confirmados.index)

In [ ]:
len(_)

Hay varias diferencias entre los países incluidos en los dataframes. Por ejemplo: en un dataframe se usa el caracter '&' y en otros la palabra 'and' para nombrar a ciertos países. Además, hay diferencias semánticas y políticas en lo que se considera en la columna de los países.

En base a estas diferencias implementamos un procedimiento ad hoc para hacer que los países de ambos dataframes coincidan.

In [ ]:
# Cambio de algunos símbolos
symbolChanges = [
    ['&', 'and'],
    ['St.', 'Saint'],
    ['ô', 'o']
]
for change in symbolChanges:
    df_poblacion.index = [country.replace(change[0], change[1]) for country in df_poblacion.index]


# Cambiamos valores específicos
fullChanges = [
    ['US','United States'],
    ['Burma','Myanmar'],
    ['Korea, South','South Korea'],
    ['Taiwan*','Taiwan'],
    ['Congo (Kinshasa)','DR Congo'],
    ['Congo (Brazzaville)','Congo'],
    ['Czechia','Czech Republic (Czechia)'],
    ['Saint Vincent and the Grenadines','Saint Vincent and Grenadines']
]

for change in fullChanges:  
    df_poblacion.index = [change[0] if country == change[1] else country for country in df_poblacion.index]


# Incluimos sectores dentro de otros ( sumando población )
includeChanges = [
    ['France','Wallis and Futuna'],
    ['France','New Caledonia'],
    ['France','French Guiana'],
    ['France','French Polynesia'],
    ['France','Guadeloupe'],
    ['France','Martinique'],
    ['France','Mayotte'],
    ['France','Réunion'],
    ['France','Saint Barthelemy'],
    ['France','Saint Pierre and Miquelon'],
    ['Denmark','Faeroe Islands'],
    ['Denmark','Greenland'],
    ['US','American Samoa'],
    ['US','Guam'],
    ['US','Northern Mariana Islands'],
    ['US','Puerto Rico'],
    ['US','U.S. Virgin Islands'],
    ['Israel','State of Palestine'],
    ['United Kingdom','Falkland Islands'],
    ['United Kingdom','Anguilla'],
    ['United Kingdom','Bermuda'],
    ['United Kingdom','British Virgin Islands'],
    ['United Kingdom','Cayman Islands'],
    ['United Kingdom','Channel Islands'],
    ['United Kingdom','Gibraltar'],
    ['United Kingdom','Isle of Man'],
    ['United Kingdom','Montserrat'],
    ['United Kingdom','Saint Helena'],
    ['United Kingdom','Turks and Caicos'],
    ['Netherlands','Aruba'],
    ['Netherlands','Caribbean Netherlands'],
    ['Netherlands','Curaçao'],
    ['Netherlands','Sint Maarten'],
    ['New Zealand','Cook Islands'],
    ['New Zealand','Niue'],
    ['New Zealand','Tokelau'],
    ['China','Hong Kong'],
    ['China','Macao'],
    ['Micronesia','Nauru'],
    ['Micronesia','Palau'],
    ['Morocco','Western Sahara']
]

for change in includeChanges:  
    df_poblacion.index = [change[0] if country == change[1] else country for country in df_poblacion.index]

# Suma población de lugares unidos
df_poblacion = df_poblacion.groupby(df_poblacion.index).sum()

df_poblacion.head(5)

3. Haga un merge de los DataFrame anteriores. El objetivo es asignar un valor de Population a cada elemento del primer DataFrame. Indique las filas donde no se puede realizar el merge y luego descártelas de su DataFrame final

In [ ]:
# Elimina lugares en df de población
removePop = ['Kiribati', 'North Korea', 'Saint Martin', 'Tonga', 'Turkmenistan', 'Tuvalu']
df_poblacion = df_poblacion.drop(removePop,axis=0).sort_index()
df_poblacion.head(5)

In [ ]:
# Elimina lugares en df de confirmados
removeCon = ['Diamond Princess', 'Kosovo', 'MS Zaandam', 'West Bank and Gaza']
df_confirmados = df_confirmados.drop(removeCon,axis=0).sort_index()
df_confirmados.head(5)

In [ ]:
# Incluye la población al df de confirmados
df_confirmados.insert(0, 'Population', df_poblacion['Population (2020)'])
df_confirmados.head(5)

4. Repita el procedimiento anterior para los archivos covid19_recuperados.csv y covid19_muertes.csv

In [ ]:
# Importamos datasets
df_recuperados = pd.read_csv('./data/covid19_recuperados.csv',index_col='Country/Region')
df_muertes = pd.read_csv('./data/covid19_decesos.csv',index_col='Country/Region')

# Elimina columnas Lat, Long y Province/State
df_recuperados = df_recuperados.drop(['Lat','Long','Province/State'],axis=1)
df_muertes = df_muertes.drop(['Lat','Long','Province/State'],axis=1)

# Convierte nombre de columnas a datetime64
df_recuperados.columns = pd.to_datetime(df_recuperados.columns).tolist()
df_muertes.columns = pd.to_datetime(df_muertes.columns).tolist()

# Compacta las filas del mismo país en una y suma sus valores
df_recuperados = df_recuperados.groupby("Country/Region").sum()
df_muertes = df_muertes.groupby("Country/Region").sum()

# Elimina Filas Conflictivas
df_recuperados = df_recuperados.drop(removeCon,axis=0).sort_index()
df_muertes = df_muertes.drop(removeCon,axis=0).sort_index()

# Incluye la población
df_recuperados.insert(0, 'Population', df_poblacion['Population (2020)'])
df_muertes.insert(0, 'Population', df_poblacion['Population (2020)'])

In [ ]:
# Verifica recuperados
df_recuperados.head(5)

In [ ]:
# Verifica muertes
df_muertes.head(5)

## Análisis global (1.5pt)
1. Obtenga la cantidad total a la fecha (26 de Abril) de los casos (a) confirmados, (b) cerrados (recuperados más decesos) y (c) activos (confirmados menos cerrados) por país. En base a estas cantidades calcule el porcentaje de
    - casos activos con respecto al total de confirmados
    - decesos con respecto al total de casos cerrados
    - recuperados con respecto al total de casos cerrados

En los archivos csv, varios países reportan información que podría no ser fidedigna.

Por ejemplo, en Suecia ('Sweden') se reportan cero recuperados para todos los días. En otros países, como Estados Unidos ('US'), se reportan cantidades decrecientes de recuperados, lo cual no tiene sentido, ya que los datos supuestamente corresponden a la cantidad acumulada de recuperados. Algo similar ocurre tambien en los archivos de casos confirmados y de muertes

Escribimos un procedimiento que modifica los tres dataframes para que no existan deltas negativos al avanzar en el tiempo.

Para un determinado día, la información después de corregir los dataframes será el valor máximo que se ha alcanzado considerando ese día y todos los anteriores.

In [ ]:
# Reemplazamos deltas negativos por el máximo anterior

for row in range(df_confirmados.shape[0]):
    for col in range(2,df_confirmados.shape[1]):
        if df_confirmados.iloc[row,col] < df_confirmados.iloc[row,col-1]:
            df_confirmados.iloc[row,col] = df_confirmados.iloc[row,col-1]
        if df_recuperados.iloc[row,col] < df_recuperados.iloc[row,col-1]:
            df_recuperados.iloc[row,col] = df_recuperados.iloc[row,col-1]
        if df_muertes.iloc[row,col] < df_muertes.iloc[row,col-1]:
            df_muertes.iloc[row,col] = df_muertes.iloc[row,col-1]

In [ ]:
# Totales

# Calcula casos confirmados globales hasta el 26 de abril
total_confirmados = df_confirmados[df_confirmados.columns[-1]]

# Calcula casos recuperados globales hasta el 26 de abril
total_recuperados = df_recuperados[df_recuperados.columns[-1]]

# Calcula decesos globales hasta el 26 de abril
total_muertes = df_muertes[df_muertes.columns[-1]]

# Calcula casos cerrados (recuperados más decesos) hasta el 26 de abril
total_cerrados = total_recuperados + total_muertes

# Calcula casos activos (confirmados menos cerrados) hasta el 26 de abril
total_activos = total_confirmados - total_cerrados

# Porcentajes

# Porcentaje de casos activos con respecto al total de confirmados
p_activos_confirmados = 100.0 * total_activos.astype('float64')/total_confirmados.astype('float64')

# Porcentaje de decesos con respecto al total de casos cerrados
p_muertes_cerrados = 100.0 * total_muertes.astype('float64')/total_cerrados.astype('float64')

# Porcentaje de recuperados con respecto al total de casos cerrados
p_recuperados_cerrados = 100.0 * total_recuperados.astype('float64')/total_cerrados.astype('float64')

2. Luego calcule las siguientes tasas en base a información a la fecha
    - Tasa de incidencia: confirmados por 100.000 dividido población total
    - Tasa de recuperación: recuperados por 100.000 dividido población total
    - Tasa de mortalidad: decesos por 100.000 dividido población total

In [ ]:
# Tasas

# Población por país (double)
poblacion_total = df_poblacion['Population (2020)'].astype('float64')

# Tasa de incidencia
tasa_incidencia = (100000.0 * total_confirmados)/poblacion_total

# Tasa de recuperación
tasa_recuperacion = (100000.0 * total_recuperados)/poblacion_total

# Tasa de mortalidad
tasa_mortalidad = (100000.0 * total_muertes)/poblacion_total

In [ ]:
recuperados_cero = ['US', 'Sweden', 'Serbia', 'Belgium']
recuperados_decrecientes = []

3. En base a lo anterior muestre los nombres y los valores de los 5 países con
    - La mayor y menor tasa de incidencia, respectivamente
    - La mayor y menor tasa de recuperación, respectivamente
    - La mayor y menor tasa de mortalidad, respectivamente

In [ ]:
# Ordenamos en orden creciente cada df de tasas
tasa_incidencia_sorted = tasa_incidencia.sort_values()
tasa_recuperacion_sorted = tasa_recuperacion.sort_values()
tasa_mortalidad_sorted = tasa_mortalidad.sort_values()

# Obtenemos 5 mínimos en cada caso
tasa_incidencia_mins = tasa_incidencia_sorted[:5]
tasa_recuperacion_mins = tasa_recuperacion_sorted[:9] # Primeros 4 no son confiables
tasa_mortalidad_mins = tasa_mortalidad_sorted[:5]

# Obtenemos 5 máximos en cada caso
tasa_incidencia_maxs = tasa_incidencia_sorted[-5:]
tasa_recuperacion_maxs = tasa_recuperacion_sorted[-5:]
tasa_mortalidad_maxs = tasa_mortalidad_sorted[-5:]

In [ ]:
# Muestra min y max tasas de incidencia
print("Paises con tasas de incidencia mínima:")
display(tasa_incidencia_mins)
print("Paises con tasas de incidencia máxima:")
display(tasa_incidencia_maxs)

In [ ]:
# Muestra min y max tasas de recuperacion
print("Paises con tasas de recuperación mínima:")
display(tasa_recuperacion_mins)
print("Paises con tasas de recuperación máxima:")
display(tasa_recuperacion_maxs)

Serbia, Bélgica, Suecia y Estados Unidos reportan cero recupardos, y estimamos que esa información podría no ser verídica.

In [ ]:
# Muestra min y max tasas de mortalidad
print("Paises con tasas de mortalidad mínima:")
display(tasa_mortalidad_mins)
print("Paises con tasas de mortalidad máxima:")
display(tasa_mortalidad_maxs)

4. Responda, analice y discuta: ¿En qué lugar/ranking se encuentra Chile con respecto a cada una de las métricas de los puntos anteriores? ¿A qué países de la OCDE se parece más? ¿A qué paises de latinoamérica se parece más?

In [ ]:
paises_ocde = ['Australia', 'Austria', 'Belgium', 'Canada', 'Chile', 'Colombia',
               'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany',
               'Greece', 'Hungary', 'Iceland', 'Ireland', 'Israel', 'Italy',
               'Japan', 'Korea, South', 'Latvia', 'Lithuania', 'Luxembourg',
               'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Poland',
               'Portugal', 'Slovakia', 'Slovenia', 'Spain', 'Sweden',
               'Switzerland', 'Turkey', 'US', 'United Kingdom']

paises_latam = ['Argentina', 'Argentina', 'Bolivia', 'Brazil', 'Chile',
                'Colombia', 'Costa Rica', 'Ecuador', 'El Salvador',
                'Guatemala', 'Guyana', 'Honduras', 'Mexico', 'Nicaragua',
                'Panama', 'Paraguay', 'Peru', 'Suriname', 'Uruguay',
                'Venezuela']

In [ ]:
def mostrar_ranking_tasa(tipo, tasa_sorted):
    tasa_ocde_sorted = tasa_sorted.loc[paises_ocde].sort_values()
    tasa_latam_sorted = tasa_sorted.loc[paises_latam].sort_values()
    print('Resumen datos de', tipo, 'a nivel internacional:',
          tasa_sorted.describe(), '\n')
    print('Resumen datos de', tipo, 'entre países de la OCDE:',
          tasa_ocde_sorted.describe(), '\n')
    print('Resumen datos de', tipo, 'entre países latinoamericanos:',
          tasa_latam_sorted.describe(), '\n')
    print('Tasa de', tipo, 'en Chile:', tasa_sorted.at['Chile'])
    print('Puesto de Chile en el ranking mundial de', tipo + ":",
          str(tasa_sorted.index.get_loc('Chile')) + '/' +
          str(len(tasa_incidencia_sorted.index)))
    print('Puesto de Chile entre países de la OCDE en', tipo + ":",
          str(tasa_ocde_sorted.index.get_loc('Chile')) + '/' +
          str(len(tasa_ocde_sorted.index)))
    print('Puesto de Chile entre países de latinoamérica en', tipo + ":",
          str(tasa_latam_sorted.index.get_loc('Chile')) + '/' +
          str(len(tasa_latam_sorted.index)), ' \n')


In [ ]:
mostrar_ranking_tasa('incidencia', tasa_incidencia_sorted)
mostrar_ranking_tasa('mortalidad', tasa_mortalidad_sorted)
mostrar_ranking_tasa('recuperación', tasa_recuperacion_sorted)

A nivel mundial, Chile se encuentra en puestos bastante bajos en incidencia, mortalidad y recuperación.

A nivel de la OCDE, se encuentra en puestos intermedios.

### Análisis temporal y comparativo (2.0pt)
1. Construya una interfaz gráfica usando ipywidgets y matplotlib que permita realizar comparaciones entre las series de tiempo de COVID-19 de dos países seleccionados arbitrariamente. A modo de simplificación considere sólo los países de la OCDE. Requerimientos:
    - La interfaz debe incluir controles para escoger dos países a partir de una lista
    - La interfaz debe incluir tres tabs, el primero debe mostrar las series de tiempo de ambos países de los casos confirmados, el segundo la de recuperados y el tercero la de los decesos, respectivamente. Use una leyenda para indicar el nombre de los países.
    - La interfaz debe tener un control que permita escoger entre casos acumulados y casos nuevos
    - La interfaz debe tener un control que permita escoger entre valores absolutos y valores relativos (tasas por 100.000 habitantes)
    - La interfaz debe tener un control que permita escoger entre valores diarios y valores semanales (acumulados). Para esto último se recomienda usar groupby con una frecuencia lunes a lunes

In [ ]:
# UI Setup

# Dropdowns

dropdownA = widgets.Dropdown(
    options=paises_ocde,
    value='Canada',
    description='Países',
    style = {'description_width':'102px'},
    layout={'width': '310px'}
)

dropdownB = widgets.Dropdown(
    options=paises_ocde,
    value='Chile',
    description='',
    layout={'width': '200px'}
)

dropdownsLayout = widgets.HBox([dropdownA,dropdownB])

# Toggles
toggleStyles = {'button_width':'200px','button_width':'100px'}

modoToggle = widgets.ToggleButtons(
    options=['Casos Diarios','Casos Acumulados'],
    description='Modo:',
    button_style='success',
    style={'button_width':'200px','description_width':'100px'},
    tooltips=['Muestra los casos nuevos de cada día o semana','Muestra los casos acumulados hasta la fecha']
)

normToggle = widgets.ToggleButtons(
    options=['Valor Absoluto', 'Valores Relativos'],
    description='Normalización:',
    button_style='info',
    style = modoToggle.style,
    tooltips=['Valor absoluto', 'Tasas por 100.000 habitantes']
)

freqToggle = widgets.ToggleButtons(
    options=['Valores Diarios', 'Valores Semanales'],
    description='Agrupamiento:',
    button_style='warning',
    style = modoToggle.style,
    tooltips=['Muestra valores diarios', 'Agrupa valores por semana ( de lunes a lunes )']
)

# Layout principal
mainLayout = widgets.VBox([dropdownsLayout,modoToggle,normToggle,freqToggle])

# Tabs
tabs = widgets.Tab()
tab_names = ['Confirmados','Recuperados','Decesos']

for i in range(3):
    tabs.set_title(i, tab_names[i])
    
tabs.children = [mainLayout,mainLayout,mainLayout]

# Layout con Output
out = widgets.Output()
outLayout = widgets.VBox([tabs,out])

In [ ]:
# Arreglo de tiempo por día
def plot():
    with out:
        clear_output()
        plt.close()
        plt.figure(figsize=(9, 5))
        
        df = None
        
        # Confirmados
        if tabs.selected_index == 0:
            df = df_confirmados.loc[paises_ocde]
            plt.title('Confirmados')
        # Recuperados
        elif tabs.selected_index == 1:
            df = df_recuperados.loc[paises_ocde]
            plt.title('Recuperados')
        # Decesos
        else:
            df = df_muertes.loc[paises_ocde]
            plt.title('Decesos')
        
        
        dfA = df.loc[dropdownA.value].copy()[1:]
        dfB = df.loc[dropdownB.value].copy()[1:]
        
        if modoToggle.value == 'Casos Diarios':
            dfA[1:] = [dfA[i]-dfA[i-1]for i in range(1,len(dfA))]
            dfB[1:] = [dfB[i]-dfB[i-1]for i in range(1,len(dfB))]
            
        if normToggle.value == 'Valores Relativos':
            dfA = (100000.0*dfA)/poblacion_total[dropdownA.value]
            dfB = (100000.0*dfB)/poblacion_total[dropdownB.value]
            plt.ylabel('Tasa por 100.000 habitantes')
        else:
            plt.ylabel('Número de Casos')
            
        if freqToggle.value == 'Valores Semanales':
            dfA = dfA.T.groupby(pd.Grouper(freq='W-MON')).sum()
            t = dfA.index.to_numpy()
            dfA = dfA.to_numpy()
            dfB = dfB.T.groupby(pd.Grouper(freq='W-MON')).sum()
            plt.xlabel('Semana')
        else:
            t = df_confirmados.columns.to_numpy()[1:]
            plt.xlabel('Día')
        
        
        plt.plot_date(t, dfA, 'r', label=dropdownA.value)
        plt.plot_date(t, dfB, 'b', label=dropdownB.value)
        plt.legend()
        plt.show()

In [ ]:
# Interacción
display(outLayout)
    
# Actualiza series
def updateSeries(a,b,c,d,e):
    plot()
    
prevIndex = 0
def updateTab():
    global prevIndex
    if prevIndex != tabs.selected_index:
        plot()
    prevIndex = tabs.selected_index
    
widgets.interactive_output(updateSeries,{
    'a':dropdownA,
    'b':dropdownB,
    'c':modoToggle,
    'd':normToggle,
    'e':freqToggle
})
tabs.observe(lambda change: updateTab(), names='selected_index')

En base a su interfaz:

1. Describa en detalle la situación de Chile con respecto a la evolución temporal de las series de tiempo. En particular destaque patrones y comportamientos que considere relevantes y compare lo que observa en las distintas series de tiempo. Considerando la evolución a nivel de semana, conteste ¿En qué fechas ocurre un aumento o disminución notoria de los casos? ¿En qué fechas hay máximos y mínimos?
2. Considerando métricas que sean relativas compare el caso de Chile contra cinco países de la OCDE seleccionados por ustedes. Destaque semejanzas y diferencias considerando las cantidades y los posibles desfases temporales existentes.